In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image

In [33]:
from ultralytics import YOLO

In [34]:
base_dir = Path("data")
import os
os.listdir(base_dir)

['classes.json',
 'classes_with_dont_care.json',
 'labels',
 'labels_with_dont_care',
 'testing',
 'training']

In [35]:
img_path= base_dir / 'training' / 'image_2'
label_path =base_dir / 'labels'
label_path,img_path

(WindowsPath('data/labels'), WindowsPath('data/training/image_2'))

In [36]:
with open(base_dir/'classes.json',"r") as f:
    classes=json.load(f)
    
classes

{'Car': 0,
 'Pedestrian': 1,
 'Van': 2,
 'Cyclist': 3,
 'Truck': 4,
 'Misc': 5,
 'Tram': 6,
 'Person_sitting': 7}

In [37]:
images = list(img_path.glob("*.png"))
labels = list(label_path.glob("*.txt"))
pairs=list(zip(images, labels))
pairs[:3]

[(WindowsPath('data/training/image_2/000000.png'),
  WindowsPath('data/labels/000000.txt')),
 (WindowsPath('data/training/image_2/000001.png'),
  WindowsPath('data/labels/000001.txt')),
 (WindowsPath('data/training/image_2/000002.png'),
  WindowsPath('data/labels/000002.txt'))]

In [38]:
from sklearn.model_selection import train_test_split
train_pairs, val_pairs = train_test_split(pairs, test_size=0.1, random_state=42)
len(train_pairs), len(val_pairs)

(6732, 749)

# Convert Seperate Vehicle Detection Dataset to YOLO Format
```
/kaggle/working
    |
    -train
    |   |
    |   -000000.png
    |   -000000.txt
    |   ...
    |
    -val
      |
      -000001.png
      -000001.txt
      ...
```

In [39]:
train_path = Path('train').resolve()
train_path.mkdir(exist_ok=True)
valid_path = Path('valid').resolve()
valid_path.mkdir(exist_ok=True)

In [40]:
for t_img, t_lb in tqdm(train_pairs):
    im_path = train_path / t_img.name
    lb_path = train_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

  0%|          | 0/6732 [00:00<?, ?it/s]

In [41]:
for t_img, t_lb in tqdm(val_pairs):
    im_path = valid_path / t_img.name
    lb_path = valid_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

  0%|          | 0/749 [00:00<?, ?it/s]

In [42]:
yaml_content = f"""
names:
- Car
- Pedestrian
- Van
- Cyclist
- Truck
- Misc
- Tram
- Person_sitting
nc: 8
train: train
val: valid
"""
with open('data.yaml', 'w') as f:
    f.write(yaml_content)

In [43]:
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

In [ ]:
# !yolo train model=/kaggle/working/yolo12s.pt  \
#     data="/kaggle/input/kitti-dataset-yolo-format/Kitti Dataset Yolo Format/data.yaml" \
#     epochs=50 batch=32 \
#     degrees=15 translate=0.1 scale=0.5 shear=5 perspective=0.0005 \
#     flipud=0.2 fliplr=0.5 hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 \
#     mosaic=1.0 mixup=0.2 copy_paste=0.2 workers=4 device=0 \
#     project=/kaggle/working/output \
#     name=trained_model \
#     save=True

In [ ]:
# Load the trained model
trained_model = YOLO('/kaggle/working/output/trained_model/weights/best.pt')

In [ ]:
# Make prediction on a sample image
sample_image_path = '/kaggle/input/kitti-dataset-yolo-format/Kitti Dataset Yolo Format/images/test_image.jpg'  # Replace with your image path

results = trained_model.predict(source=sample_image_path, conf=0.5)

# Display results
for result in results:
    print(f"Detections found: {len(result.boxes)}")
    print(f"Classes: {result.names}")
    print(f"Confidence scores: {result.boxes.conf}")
    print(f"Bounding boxes: {result.boxes.xyxy}")
    
    # Display image with predictions
    result.show()

In [ ]:
# Alternative: Batch prediction on multiple images
from pathlib import Path

image_folder = '/kaggle/input/kitti-dataset-yolo-format/Kitti Dataset Yolo Format/images/test/'
image_files = list(Path(image_folder).glob('*.jpg'))

# Predict on all images
batch_results = trained_model.predict(source=image_files, conf=0.5)

# Process results
for i, result in enumerate(batch_results):
    print(f"\nImage {i+1}: {result.path}")
    print(f"Detections: {len(result.boxes)}")
    for j, box in enumerate(result.boxes):
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        class_name = result.names[class_id]
        print(f"  Object {j+1}: {class_name} (confidence: {confidence:.2f})")